In [1]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn
import os, pathlib, shutil, sys
import kaggle, kagglehub

print(
    "python " + sys.version.split()[0],
    "numpy " + np.__version__,
    "polars " + pl.__version__,
    "scikit-learn " + sklearn.__version__,
    sep="\n",
)

python 3.13.1
numpy 2.2.2
polars 1.21.0
scikit-learn 1.6.1


### Downloading the competition

In [2]:
competition_name = "store-sales-time-series-forecasting"
competition_path = pathlib.Path(
    kagglehub.competition.competition_download(competition_name)
)
competition_files: list = os.listdir(competition_path)

print(competition_files)

shutil.copytree(competition_path, "./data", dirs_exist_ok=True)

['holidays_events.csv', 'oil.csv', 'sample_submission.csv', 'stores.csv', 'test.csv', 'train.csv', 'transactions.csv']


'./data'

Load all csv files of the competition into polars dataframes

In [3]:
# Load the csv files `['holidays_events.csv', 'oil.csv', 'sample_submission.csv', 'stores.csv', 'test.csv', 'train.csv', 'transactions.csv']` into polars dataframes
train_df = pl.read_csv("./data/train.csv")
test_df = pl.read_csv("./data/test.csv")
sample_submission_df = pl.read_csv("./data/sample_submission.csv")
holidays_events_df = pl.read_csv("./data/holidays_events.csv")
oil_df = pl.read_csv("./data/oil.csv")
stores_df = pl.read_csv("./data/stores.csv")
transactions_df = pl.read_csv("./data/transactions.csv")


Cast the dtype of date cols to `pl.Date`

In [4]:
# date included dfs list:
dfs: list[pl.DataFrame] = [
    train_df,
    test_df,
    holidays_events_df,
    oil_df,
    transactions_df,
]
dfs = [df.with_columns(pl.col("date").cast(pl.Date)) for df in dfs]
train_df, test_df, holidays_events_df, oil_df, transactions_df = dfs

In [5]:
# number of unique values
print("train_df unique values: \n", train_df.select(pl.col("*").n_unique()))
print("test_df unique values: \n", test_df.select(pl.col("*").n_unique()))
print(
    "holidays_events_df unique values: \n",
    holidays_events_df.select(pl.col("*").n_unique()),
)
print("oil_df unique values: \n", oil_df.select(pl.col("*").n_unique()))
print(
    "transactions_df unique values: \n",
    transactions_df.select(pl.col("*").n_unique()),
)
print("stores_df unique values: \n", stores_df.select(pl.col("*").n_unique()))


train_df unique values: 
 shape: (1, 6)
┌─────────┬──────┬───────────┬────────┬────────┬─────────────┐
│ id      ┆ date ┆ store_nbr ┆ family ┆ sales  ┆ onpromotion │
│ ---     ┆ ---  ┆ ---       ┆ ---    ┆ ---    ┆ ---         │
│ u32     ┆ u32  ┆ u32       ┆ u32    ┆ u32    ┆ u32         │
╞═════════╪══════╪═══════════╪════════╪════════╪═════════════╡
│ 3000888 ┆ 1684 ┆ 54        ┆ 33     ┆ 379610 ┆ 362         │
└─────────┴──────┴───────────┴────────┴────────┴─────────────┘
test_df unique values: 
 shape: (1, 5)
┌───────┬──────┬───────────┬────────┬─────────────┐
│ id    ┆ date ┆ store_nbr ┆ family ┆ onpromotion │
│ ---   ┆ ---  ┆ ---       ┆ ---    ┆ ---         │
│ u32   ┆ u32  ┆ u32       ┆ u32    ┆ u32         │
╞═══════╪══════╪═══════════╪════════╪═════════════╡
│ 28512 ┆ 16   ┆ 54        ┆ 33     ┆ 212         │
└───────┴──────┴───────────┴────────┴─────────────┘
holidays_events_df unique values: 
 shape: (1, 6)
┌──────┬──────┬────────┬─────────────┬─────────────┬─────────────┐

In [6]:
all_dfs: list[pl.DataFrame] = [
    train_df,
    test_df,
    holidays_events_df,
    oil_df,
    transactions_df,
    stores_df,
]

[df.glimpse(max_items_per_column=0) for df in all_dfs]

Rows: 3000888
Columns: 6
$ id           <i64> 
$ date        <date> 
$ store_nbr    <i64> 
$ family       <str> 
$ sales        <f64> 
$ onpromotion  <i64> 

Rows: 28512
Columns: 5
$ id           <i64> 
$ date        <date> 
$ store_nbr    <i64> 
$ family       <str> 
$ onpromotion  <i64> 

Rows: 350
Columns: 6
$ date        <date> 
$ type         <str> 
$ locale       <str> 
$ locale_name  <str> 
$ description  <str> 
$ transferred <bool> 

Rows: 1218
Columns: 2
$ date       <date> 
$ dcoilwtico  <f64> 

Rows: 83488
Columns: 3
$ date         <date> 
$ store_nbr     <i64> 
$ transactions  <i64> 

Rows: 54
Columns: 5
$ store_nbr <i64> 
$ city      <str> 
$ state     <str> 
$ type      <str> 
$ cluster   <i64> 



[None, None, None, None, None, None]

In [36]:
import polars as pl
from functools import reduce
from collections import Counter

# List your dataframes
all_dfs: list[pl.DataFrame] = [
    train_df,
    test_df,
    holidays_events_df,
    oil_df,
    transactions_df,
    stores_df,
]

# Get the set of columns for each dataframe
df_cols = [set(df.columns) for df in all_dfs]

# Count how many times each column appears
col_counts = Counter(col for cols in df_cols for col in cols)

print(col_counts.most_common(6))


[('date', 5), ('store_nbr', 4), ('onpromotion', 2), ('family', 2), ('id', 2), ('type', 2)]


In [11]:
date_included_dfs: list[pl.DataFrame] = [
    train_df,
    holidays_events_df,
    oil_df,
    transactions_df,
]